**ResNet** model implementation on CIFAR10 with +91% accuracy

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/CS345/CS345Project_Part3"

Mounted at /content/gdrive/
--------------------------------------------------------------------------------
ResNet_for_CIFAR_10_with_Keras.ipynb


In [ ]:
import os

os.chdir("/content/gdrive/My Drive/Colab Notebooks/CS345/CS345Project_Part3")
print("Current dir: ", os.getcwd())

Current dir:  /content/gdrive/My Drive/Colab Notebooks/CS345/CS345Project_Part3


In [ ]:
from __future__ import absolute_import 
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import math

In [ ]:
# training parameters
batch_size = 30
epochs = 200
data_augmentation = True
num_classes = 10

# subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# n decides the calculated depth of the model
n = 9
depth = n * 6 + 2
version = 1
# model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
def lr_schedule(epoch): #method that brings the learning rate down as the number of epochs increases
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


#function that creates a resnet layer including the two normal layers and the skip connection
#returns a tensor as an input to the next layer
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True, conv_first=True):
    
    #the Conv2D layer
    conv = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))

    #batch normalization input that helps with the vanishing gradient issue
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
def resnet(input_shape, depth, num_classes=10):
   
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')

    # start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape) #the input parameter specifies the shape
    x = resnet_layer(inputs=inputs)

    # stacking up the resnet layers to create the blocks
    for stack in range(3):
        for res_block in range(num_res_blocks): #the resnet block 
            strides = 1
            # first layer but not first stack
            if stack > 0 and res_block == 0:  
                strides = 2  # downsample

            y = resnet_layer(inputs=x, num_filters=num_filters, strides=strides) #first normal layer of the block keeping track of the x as input
            y = resnet_layer(inputs=y, num_filters=num_filters, activation=None) #second normal layer of the block keeping track of the y as input

            # first layer but not first stack
            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, num_filters=num_filters, kernel_size=1, strides=strides, activation=None, batch_normalization=False)
            x = add([x, y]) #the skip layer
            x = Activation('relu')(x)
        num_filters *= 2

    # add classifier on top
    x = AveragePooling2D(pool_size=8)(x) #the averaging pool
    y = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(y) #the dense layer

    # instantiate model
    model = Model(inputs=inputs, outputs=outputs)
    return model




Running the model

In [ ]:
# input image dimensions
input_shape = x_train.shape[1:]

# data normalization
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if enabled, substract the mean
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [ ]:
model = resnet(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['acc'])
model.summary()
print(model_type)

# saving model to the specified directory
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
________________________________________________________________________

In [ ]:
# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule) 
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6) #reduces the learning rate when it plateaus 

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [ ]:
# run training, with or without data augmentation

#if not true
if not data_augmentation:
    print('Not using data augmentation.')

    #normal fit
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True, callbacks=callbacks)
else:

    #prepocessing and realtime data augmentation to reduce overfit:
    print('Using real-time data augmentation.')

    datagen = ImageDataGenerator(
        featurewise_center=False,            # set input mean to 0 over the dataset
        samplewise_center=False,             # set each sample mean to 0
        featurewise_std_normalization=False, # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,                 # apply ZCA whitening
        rotation_range=0,                    # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.1,               # randomly shift images horizontally
        height_shift_range=0.1,              # randomly shift images vertically
        horizontal_flip=True,                # randomly flip images
        vertical_flip=False)                 # randomly flip images

    datagen.fit(x_train)

    # calculating the steps of each epoch
    steps_per_epoch =  math.ceil(len(x_train) / batch_size)

    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size), verbose=1, epochs=epochs, validation_data=(x_test, y_test), steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)


Using real-time data augmentation.
Epoch 1/200
Learning rate:  0.001
1667/1667 [==============================] - 98s 37ms/step - loss: 2.3719 - acc: 0.3682 - val_loss: 1.8118 - val_acc: 0.5272

Epoch 00001: val_acc improved from -inf to 0.52720, saving model to /content/gdrive/My Drive/Colab Notebooks/CS345/CS345Project_Part3/saved_models/cifar10_ResNet56v1_model.001.h5
Epoch 2/200
Learning rate:  0.001
1667/1667 [==============================] - 61s 37ms/step - loss: 1.4587 - acc: 0.6069 - val_loss: 1.4520 - val_acc: 0.6171

Epoch 00002: val_acc improved from 0.52720 to 0.61710, saving model to /content/gdrive/My Drive/Colab Notebooks/CS345/CS345Project_Part3/saved_models/cifar10_ResNet56v1_model.002.h5
Epoch 3/200
Learning rate:  0.001
1667/1667 [==============================] - 61s 37ms/step - loss: 1.2140 - acc: 0.6844 - val_loss: 1.4128 - val_acc: 0.6360

Epoch 00003: val_acc improved from 0.61710 to 0.63600, saving model to /content/gdrive/My Drive/Colab Notebooks/CS345/CS345P

In [ ]:
# score trained model
scores = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.4268166422843933
Test accuracy: 0.9210000038146973
